In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import copy
import cv2
import json
from tqdm import tqdm

import matplotlib.pyplot as plt

from IPython.display import Image as img
from IPython.display import display

from PIL import Image

import torch
import torchvision

In [ ]:
from dass_det.models.yolox import YOLOX
from dass_det.models.yolo_head import YOLOXHead
from dass_det.models.yolo_head_stem import YOLOXHeadStem
from dass_det.models.yolo_pafpn import YOLOPAFPN

from dass_det.data.data_augment import ValTransform
from dass_det.data.datasets.manga109 import Manga109Dataset
from dass_det.data.datasets.icartoonface import ICartoonFaceDataset
from dass_det.data.datasets.comic2k import Comic2kDataset
from dass_det.data.datasets.ebdtheque import EBDthequeDataset
from dass_det.data.datasets.dcm772 import DCM772Dataset

from dass_det.evaluators.comic_evaluator import ComicEvaluator

from dass_det.utils import (
    postprocess,
    xyxy2xywh,
    vis
)

## Set the Parameters Below

In [ ]:
model_path = "weights/sample_model.pth" # None # "weights/..."
model_size = "xs" # "xl"
model_mode = 0    # 1 for only face, 2 for only body

nms_thold  = 0.65
conf_thold = 0.01

data_paths = {
    "icf"        : "/datasets/iCartoonFace2020/",
    "icf_detval" : "/userfiles/comics_grp/datasets/icf_val_annot/personai_icartoonface_detval.csv",
    "m109"       : "/datasets/manga109/",
    "dcm"        : "/userfiles/comics_grp/datasets/dcm772/dcm-dataset_from_rigaud/",
    "c2k"        : "/userfiles/comics_grp/datasets/comic2k",
    "w2k"        : "/userfiles/comics_grp/datasets/watercolor2k",
    "c1k"        : "/userfiles/comics_grp/datasets/clipart2k",
    "ebd"        : "/userfiles/comics_grp/datasets/eBDtheque2019/eBDtheque_database_v3/",
}

data_bs = {
    "icf"       : 32,
    "m109"      : 24,
    "dcm"       : 12,
    "c2k"       : 32,
    "ebd"       : 4,
}

data_imgsize = {
    "icf"       : (640, 640),
    "m109"      : (1152, 1632),
    "dcm"       : (1600, 1600),
    "c2k"       : (640, 640),
    "ebd"       : (2048, 2048),
}

## Load Model

In [ ]:
assert model_path is not None
assert model_size in ["xs", "xl"]
assert model_mode in [0, 1, 2]

if model_size == "xs":
    depth, width = 0.33, 0.375
elif model_size == "xl":
    depth, width = 1.33, 1.25

model = YOLOX(backbone=YOLOPAFPN(depth=depth, width=width),
              head_stem=YOLOXHeadStem(width=width),
              face_head=YOLOXHead(1, width=width),
              body_head=YOLOXHead(1, width=width))

d = torch.load(model_path, map_location=torch.device('cpu'))

if "teacher_model" in d.keys():
    model.load_state_dict(d["teacher_model"])
else:
    model.load_state_dict(d["model"])
model = model.eval().cuda()

del d

## Evaluate ICartoonFace Faces

In [ ]:
dataset = ICartoonFaceDataset(data_dir={
    "icf_train_imgs": os.path.join(data_paths["icf"], "personai_icartoonface_dettrain/icartoonface_dettrain/"),
    "icf_test_imgs": os.path.join(data_paths["icf"], "personai_icartoonface_detval/"),
    "icf_train_labels": os.path.join(data_paths["icf"], "personai_icartoonface_dettrain/icartoonface_dettrain.csv"),
    "icf_test_labels": data_paths["icf_detval"],
}, train=False, img_size=data_imgsize["icf"], preproc=ValTransform())

dataloader = torch.utils.data.DataLoader(dataset, batch_size=data_bs["icf"])
dl_iter = iter(dataloader)

evaluator = ComicEvaluator(dataloader, data_imgsize["icf"], nms_thold, conf_thold, 1, use_real_size=False, verbose=True)
results = evaluator.evaluate(model, use_07_metric=False, mode=1)

## Evaluate Manga109 Pages

#### Faces

In [ ]:
dataset = Manga109Dataset(data_dir=data_paths["m109"],
                          preproc=ValTransform(),
                          train=False, img_size=data_imgsize["m109"])

dataloader = torch.utils.data.DataLoader(dataset, batch_size=data_bs["m109"])

evaluator = ComicEvaluator(dataloader, data_imgsize["m109"], nms_thold, conf_thold, 1, use_real_size=False, verbose=True)
results = evaluator.evaluate(model, use_07_metric=True, select_index=1, mode=1)

#### Bodies

In [ ]:
dataset = Manga109Dataset(data_dir=data_paths["m109"],
                          preproc=ValTransform(),
                          train=False, img_size=data_imgsize["m109"])

dataloader = torch.utils.data.DataLoader(dataset, batch_size=data_bs["m109"])

evaluator = ComicEvaluator(dataloader, data_imgsize["m109"], nms_thold, conf_thold, 1, use_real_size=False, verbose=True)
results = evaluator.evaluate(model, use_07_metric=True, select_index=2, mode=2)

## Evaluate DCM772

#### Faces

In [ ]:
dataset = DCM772Dataset(data_dir=data_paths["dcm"], 
                        preproc=ValTransform(),
                        train=False, img_size=data_imgsize["dcm"])

dataloader = torch.utils.data.DataLoader(dataset, batch_size=data_bs["dcm"])

evaluator = ComicEvaluator(dataloader, data_imgsize["dcm"], nms_thold, conf_thold, 1, use_real_size=False, verbose=True)
results = evaluator.evaluate(model, use_07_metric=False, select_index=1, mode=1)

#### Bodies with Background & Animals

In [ ]:
dataset = DCM772Dataset(data_dir=data_paths["dcm"], 
                        preproc=ValTransform(),
                        train=False, img_size=data_imgsize["dcm"])

dataloader = torch.utils.data.DataLoader(dataset, batch_size=data_bs["dcm"])

evaluator = ComicEvaluator(dataloader, data_imgsize["dcm"], nms_thold, conf_thold, 1, use_real_size=False, verbose=True)
results = evaluator.evaluate(model, use_07_metric=False, select_index=2, mode=2)

## Evaluate Comic2k

In [ ]:
dataset = Comic2kDataset(data_dir={
    "comic": data_paths["c2k"], 
    "watercolor": data_paths["w2k"], 
    "clipart": data_paths["c1k"]
}, train=False, img_size=data_imgsize["c2k"], filter="comic")

transform = ValTransform()
dataset.preproc = transform
dataloader = torch.utils.data.DataLoader(dataset, batch_size=data_bs["c2k"])

evaluator = ComicEvaluator(dataloader, data_imgsize["c2k"], nms_thold, conf_thold, 1, use_real_size=False, verbose=True)
results = evaluator.evaluate(model, use_07_metric=True, mode=2)

## Evaluate Watercolor2k

In [ ]:
dataset = Comic2kDataset(data_dir={
    "comic": data_paths["c2k"], 
    "watercolor": data_paths["w2k"], 
    "clipart": data_paths["c1k"]
}, train=False, img_size=data_imgsize["c2k"], filter="watercolor")

transform = ValTransform()
dataset.preproc = transform
dataloader = torch.utils.data.DataLoader(dataset, batch_size=data_bs["c2k"])

evaluator = ComicEvaluator(dataloader, data_imgsize["c2k"], nms_thold, conf_thold, 1, use_real_size=False, verbose=True)
results = evaluator.evaluate(model, use_07_metric=True, mode=2)

## Evaluate Clipart 2k

In [ ]:
dataset = Comic2kDataset(data_dir={
    "comic": data_paths["c2k"], 
    "watercolor": data_paths["w2k"], 
    "clipart": data_paths["c1k"]
}, train=False, img_size=data_imgsize["c2k"], filter="clipart")

transform = ValTransform()
dataset.preproc = transform
dataloader = torch.utils.data.DataLoader(dataset, batch_size=data_bs["c2k"])

evaluator = ComicEvaluator(dataloader, data_imgsize["c2k"], nms_thold, conf_thold, 1, use_real_size=False, verbose=True)
results = evaluator.evaluate(model, use_07_metric=True, mode=2)

## Evaluate eBDtheque

In [ ]:
img_size = (2048, 2048)
bs       = 4

dataset = EBDthequeDataset(data_dir={
    "imgs": os.path.join(data_paths["ebd"], "Pages/"),
    "labels": os.path.join(data_paths["ebd"], "GT/"),
}, train=False, img_size=data_imgsize["ebd"], preproc=ValTransform())
dataloader = torch.utils.data.DataLoader(dataset, batch_size=data_bs["ebd"])

evaluator = ComicEvaluator(dataloader, data_imgsize["ebd"], nms_thold, conf_thold, 1, use_real_size=False, verbose=True)
results = evaluator.evaluate(model, use_07_metric=False, mode=2, select_index=1)

# File Writer

### ICartoonFace

In [ ]:
from tqdm import tqdm

# Set these paths by yourself if you want to extract prediction csv's for original iCartoonFace evaluations
icf_path = "/datasets/iCartoonFace2020/personai_icartoonface_detval/"
save_path = f'eval_outs/icf_outs/icf_yolox_preds_{conf_thold}_{nms_thold}.csv'

files = os.listdir(icf_path)
transform = ValTransform()

f = open(save_path, "w")

for file in tqdm(files):
    imgs = cv2.imread(os.path.join(icf_path, file))
    h, w, c = imgs.shape
    resize_size = data_imgsize["icf"]

    imgs, labels = transform(imgs, None, resize_size)
    img_cu = torch.from_numpy(imgs).unsqueeze(0).cuda()
    
    with torch.no_grad():
        preds, _ = model(img_cu, None, mode=1)
        preds = postprocess(preds, 1, conf_thold, nms_thold)[0]
    
    del img_cu
    
    if preds is None:
        continue
      
    scale = min(resize_size[0] / float(h), resize_size[1] / float(w))
    preds[:,:4] /= scale
    preds[:,0]  = torch.max(preds[:,0], torch.zeros(preds.shape[0]).cuda())
    preds[:,1]  = torch.max(preds[:,1], torch.zeros(preds.shape[0]).cuda())
    preds[:,2]  = torch.min(preds[:,2], torch.zeros(preds.shape[0]).fill_(w).cuda())
    preds[:,3]  = torch.min(preds[:,3], torch.zeros(preds.shape[0]).fill_(h).cuda())
    
    for i in range(preds.shape[0]):
        x1, y1, x2, y2, conf = preds[i,:5]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        f.write(f'{file},{x1},{y1},{x2},{y2},face,{conf}\n')
f.close()